In [1]:
from funcs.WLANFuncs import *
from funcs.HelperFuncs import *

import os
from glob import glob

import warnings
warnings.filterwarnings("ignore")


In [ ]:
import pandas as pd
# df = pd.read_hdf("/Volumes/tim_details/tim_honours/CAPTURES/client_pc_200/v=3InbMow9IYo/wlan_h5/cap_0.h5", key=WLANFuncs.H5_WLAN_KEY, mode="r")
df[[
    'frame.number', 'frame.time_relative', 'frame.len',
    # 'wlan_radio.signal_dbm',
    'wlan.seq', 'wlan.frag', 'wlan.fc',
    'wlan.ta', 'wlan.ra',
]]

# Convert JSON to H5 (good for ML and similar to csv)
## Also adding derivate features
* Is upstream/downstream

In [2]:
ssid = "67Marlie"
pwd = "sayplease"

# dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")
dirs = glob("/Users/timothylee/Desktop/captures/client_pi_200/v=3InbMow9IYo")

# Iterating through each capture directory
for dir in dirs:
    # Getting directory paths
    wlan_dir = os.path.join(dir, "wlan")
    metadata_dir = os.path.join(dir, "metadata")
    json_dir = os.path.join(dir, "wlan_json")
    h5_dir = os.path.join(dir, "wlan_h5")
    # Preparing directories
    clean_dir_junk(wlan_dir)
    make_dir(json_dir)
    make_dir(h5_dir)
    # Reading each WLAN pcap, processing, and saving results
    for fp in os.listdir(wlan_dir)[:1]:
        name = get_name(fp)
        print(f"{dir} - {name}")
        # Getting file paths
        wlan_fp = os.path.join(wlan_dir, f"{name}.pcap")
        metadata_fp = os.path.join(metadata_dir, f"{name}.json")
        json_fp = os.path.join(json_dir, f"{name}.json")
        h5_fp = os.path.join(h5_dir, f"{name}.h5")

        # If the json_fp or h5_fp already exists, then skip
        # if os.path.isfile(h5_fp):
        #     continue
        
        # Getting metadata
        metadata = parse_json(metadata_fp)
        # Converting pcap to decrypted json
        pcap_to_json(wlan_fp, json_fp, ssid, pwd, TSHARK_FIELDS)
        # Converting json to h5, processing (adding attributes), and saving
        df = json_to_df(json_fp)
        df = df_wlan_clean(df, FIELDS)
        df = df_wlan_derive_attrs(df, metadata["mac"])
        df.to_hdf(h5_fp, key=H5_WLAN_KEY, mode="w")
    # Making space on the filesystem by removing the wlan_json dir (always large)
    remove_dir(json_dir)


/Users/timothylee/Desktop/captures/client_pi_200/v=3InbMow9IYo - cap_48


In [1]:
from funcs.WLANFuncs import *
from funcs.HelperFuncs import *

wlan_fp = "/Users/timothylee/Desktop/captures/client_pi_200/v=3InbMow9IYo/wlan/cap_48.pcap"
json_fp = "/Users/timothylee/Desktop/captures/client_pi_200/v=3InbMow9IYo/wlan_json/cap_48.json"

ssid = "67Marlie"
pwd = "sayplease"

# make_dir(json_dir)
pcap_to_json(wlan_fp, json_fp, ssid, pwd, TSHARK_FIELDS)

In [3]:
df = pd.read_hdf("/Users/timothylee/Desktop/captures/client_pi_200/v=3InbMow9IYo/wlan_h5/cap_48.h5", key=H5_WLAN_KEY, mode="r")

# df = df.fillna(0)

df.dtypes

frame.time_relative      float64
frame.number               int64
frame.len                  int32
wlan_radio.signal_dbm       int8
wlan.fc.version             int8
wlan.fc.type                int8
wlan.fc.subtype             int8
wlan.frag                   int8
wlan.seq                   int16
wlan.da                   object
wlan.sa                   object
ip.src                    object
ip.dst                    object
ip.len                     int16
udp.srcport                int16
udp.dstport                int16
tcp.srcport                int16
tcp.dstport                int16
is_upstream                 bool
dtype: object

In [ ]:
frame.time_relative      float64
frame.number               int64
frame.len                  int32
wlan_radio.signal_dbm       int8
wlan.frag                   int8
wlan.seq                   int16
wlan.fc                   object
wlan.da                   object
wlan.sa                   object
wlan.bssid                object
ip.src                    object
ip.dst                    object
ip.len                     int16
udp.srcport                int16
udp.dstport                int16
udp.stream                 int32
udp.length                 int16
tcp.srcport                int16
tcp.dstport                int16
tcp.stream                 int32
tcp.len                    int16
is_upstream                 bool
stream                    object
dtype: object

## Investigating streams

## Visualising cumulative bytes by time (binned in time intervals)

In [ ]:
# dirs = glob("/Users/timothylee/Desktop/Uni/Yr5/Honours/honours_dev/2_analyse/wlan_captures")
dirs = glob("/Volumes/tim_details/tim_honours/CAPTURES/*/*")

for dir in dirs:
    # Getting directory and file paths
    h5_dir = os.path.join(dir, "wlan_h5")
    plot_dir = os.path.join(dir, "wlan_plots")
    plot_fp = os.path.join(plot_dir, "all_cumsum.png")
    # Preparing directories
    make_dir(plot_dir)

    # If the plot already exists, then skip
    if os.path.isfile(plot_fp):
        continue

    # Making plots
    fig, ax = WLANFuncs.wlan_cumsum_plot(h5_dir)
    fig.savefig(plot_fp)
    fig.clf();
plt.close();